In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

order_df = pd.read_csv("order.csv") 

account_df = pd.read_csv("account.csv")

account_df.head()

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [2]:
order_df.shape

(6471, 6)

In [3]:
order_df['k_symbol'] = order_df['k_symbol'].fillna('Other')
order_df['k_symbol'].value_counts(dropna=False)
order_df.head(10)

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,Other
5,29406,3,AB,59972357,3539.0,POJISTNE
6,29407,4,UV,26693541,2078.0,SIPO
7,29408,4,UV,5848086,1285.0,SIPO
8,29409,5,GH,37390208,2668.0,SIPO
9,29410,6,AB,44486999,3954.0,SIPO


In [4]:
order_dum = pd.get_dummies(order_df[['k_symbol']])

order_all = pd.concat([order_df,order_dum],axis=1,ignore_index=False)  

order_all = order_all.drop(columns = ['order_id','bank_to','account_to','k_symbol'])

for col in order_dum.columns:
    order_all[col] = order_all[col] * order_all['amount']

order_acc = order_all.groupby('account_id').sum()

order_acc = order_acc.rename(columns={"amount": "amount_order"})

order_acc.head()

,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER
account_id,,,,,,
1,2452.0,0.0,0.0,0.0,2452.0,0.0
2,10638.7,0.0,0.0,0.0,7266.0,3372.7
3,5001.0,0.0,327.0,3539.0,1135.0,0.0
4,3363.0,0.0,0.0,0.0,3363.0,0.0
5,2668.0,0.0,0.0,0.0,2668.0,0.0


In [5]:
account_df.head(10)

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30
5,6,51,POPLATEK MESICNE,1994-09-27
6,7,60,POPLATEK MESICNE,1996-11-24
7,8,57,POPLATEK MESICNE,1995-09-21
8,9,70,POPLATEK MESICNE,1993-01-27
9,10,54,POPLATEK MESICNE,1996-08-28


In [6]:
account_df = pd.merge(account_df, order_acc, on='account_id', how='left')

account_df.head()

#Order now done.... onto to transactions

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0


In [7]:
#First half of transactions

trans_1_df = pd.read_csv("trans_1.csv",parse_dates=[1])

trans_1_df = trans_1_df.drop(columns=['trans_id','bank','account'])

trans_1_df['k_symbol'] = trans_1_df['k_symbol'].replace(r'^\s*$', np.nan, regex=True)

trans_1_df['operation'] = trans_1_df['operation'].fillna('Other')
trans_1_df['k_symbol'] = trans_1_df['k_symbol'].fillna('Other')

trans_1_df_dum = pd.get_dummies(trans_1_df[['type','operation','k_symbol']])

trans_1_df_all = pd.concat([trans_1_df,trans_1_df_dum],axis=1,ignore_index=False)  

for col in trans_1_df_dum.columns:
    trans_1_df_all[col] = trans_1_df_all[col] * trans_1_df_all['amount']

    
trans_1_df_all = trans_1_df_all.drop(columns = ['type','operation','k_symbol'])    

trans_1_df_all.head()


,account_id,date,amount,balance,type_PRIJEM,type_VYBER,type_VYDAJ,operation_Other,operation_PREVOD NA UCET,operation_PREVOD Z UCTU,operation_VKLAD,operation_VYBER,operation_VYBER KARTOU,k_symbol_DUCHOD,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SANKC. UROK,k_symbol_SIPO,k_symbol_SLUZBY,k_symbol_UROK,k_symbol_UVER
0,1,1995-04-30,19,17298,19,0,0,19,0,0,0,0,0,0,0,0,0,0,0,19,0
1,1,1995-05-31,79,23156,79,0,0,79,0,0,0,0,0,0,0,0,0,0,0,79,0
2,1,1995-06-30,101,26736,101,0,0,101,0,0,0,0,0,0,0,0,0,0,0,101,0
3,1,1995-07-31,109,25224,109,0,0,109,0,0,0,0,0,0,0,0,0,0,0,109,0
4,1,1995-08-31,99,21502,99,0,0,99,0,0,0,0,0,0,0,0,0,0,0,99,0


In [8]:
trans_acc = trans_1_df_all.groupby('account_id').agg({ 'date' : ['min','max'],
                            'amount' : ['sum','mean','std'],
                            'balance': ['min','max','mean','std'],
                            'type_PRIJEM' : ['sum'],
                            'type_VYBER': ['sum'],
                            'type_VYDAJ': ['sum'], 
                            'operation_Other': ['sum'], 
                            'operation_PREVOD NA UCET': ['sum'],
                            'operation_PREVOD Z UCTU': ['sum'], 
                            'operation_VKLAD': ['sum'],
                            'operation_VYBER': ['sum'],
                            'operation_VYBER KARTOU': ['sum'], 
                            'k_symbol_DUCHOD': ['sum'],
                            'k_symbol_Other': ['sum'], 
                            'k_symbol_POJISTNE': ['sum'], 
                            'k_symbol_SANKC. UROK': ['sum'],
                            'k_symbol_SIPO': ['sum'], 
                            'k_symbol_SLUZBY': ['sum'], 
                            'k_symbol_UROK': ['sum'], 
                            'k_symbol_UVER': ['sum']
                            })



trans_acc.columns = ["_".join(x) for x in trans_acc.columns.ravel()]

trans_acc.head()

,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0
10,1996-08-28,1998-12-31,1040615,6670.608974,7335.994095,1100,83410,47792.211538,12152.299822,549951,0,490664,5528,161759,0,544423,328905,0,0,872983,0,0,161759,345,5528,0
100,1993-12-28,1998-12-31,1691619,4176.837037,5074.897975,600,84377,42188.222222,12710.169747,883222,0,808397,10002,443912,0,873220,364485,0,0,1647304,0,0,33488,825,10002,0
1000,1993-01-25,1998-12-31,4779772,8184.541096,11288.861871,200,102396,57893.123288,18120.689825,2435539,530641,1813592,15443,502902,2413796,6300,1841331,0,0,4359262,191084,0,212993,990,15443,0
1002,1996-09-17,1998-12-31,264413,1592.849398,1841.461951,500,29756,17292.879518,5206.529453,143873,0,120540,2027,62810,0,141846,57730,0,0,223270,10340,0,28446,330,2027,0


In [9]:
#Second Half of Transactions

trans_2_df = pd.read_csv("trans_2.csv",parse_dates=[1])

trans_2_df = trans_2_df.drop(columns=['trans_id','bank','account'])

trans_2_df['k_symbol'] = trans_2_df['k_symbol'].replace(r'^\s*$', np.nan, regex=True)

trans_2_df['operation'] = trans_2_df['operation'].fillna('Other')
trans_2_df['k_symbol'] = trans_2_df['k_symbol'].fillna('Other')

trans_2_df_dum = pd.get_dummies(trans_2_df[['type','operation','k_symbol']])

trans_2_df_all = pd.concat([trans_2_df,trans_2_df_dum],axis=1,ignore_index=False)  

for col in trans_2_df_dum.columns:
    trans_2_df_all[col] = trans_2_df_all[col] * trans_2_df_all['amount']

trans_2_df_all = trans_2_df_all.drop(columns = ['type','operation','k_symbol'])    

trans_2_acc = trans_2_df_all.groupby('account_id').agg({ 'date' : ['min','max'],
                            'amount' : ['sum','mean','std'],
                            'balance': ['min','max','mean','std'],
                            'type_PRIJEM' : ['sum'],
                            'type_VYBER': ['sum'],
                            'type_VYDAJ': ['sum'], 
                            'operation_Other': ['sum'], 
                            'operation_PREVOD NA UCET': ['sum'],
                            'operation_PREVOD Z UCTU': ['sum'], 
                            'operation_VKLAD': ['sum'],
                            'operation_VYBER': ['sum'],
                            'operation_VYBER KARTOU': ['sum'], 
                            'k_symbol_DUCHOD': ['sum'],
                            'k_symbol_Other': ['sum'], 
                            'k_symbol_POJISTNE': ['sum'], 
                            'k_symbol_SANKC. UROK': ['sum'],
                            'k_symbol_SIPO': ['sum'], 
                            'k_symbol_SLUZBY': ['sum'], 
                            'k_symbol_UROK': ['sum'], 
                            'k_symbol_UVER': ['sum']
                            })



trans_2_acc.columns = ["_".join(x) for x in trans_2_acc.columns.ravel()]

trans_2_acc.head()

,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,
10001,1995-05-28,1998-12-31,1454586,5865.266129,7311.353622,600,101113,56285.205645,15714.905405,766746,0,687840,9398,15870,0,757348,671970,0,0,1428748,0,0,0,570,9398,15870
10005,1997-11-29,1998-12-31,969769,13284.506849,14843.203395,200,87393,46756.616438,18956.738748,511790,18130,439849,2276,111629,509314,200,346350,0,0,855744,0,0,82359,120,2276,29270
10018,1995-07-23,1998-12-31,1784912,5475.190184,7617.888148,-4953,68876,23585.634969,12684.444374,906592,0,878320,6760,0,0,899832,878320,0,0,1511232,0,80,266300,540,6760,0
10019,1995-06-15,1998-12-12,2606945,10427.780000,13185.943727,600,138173,52650.048000,27465.267872,1314297,199558,1093090,8120,146205,0,1306177,1146443,0,0,2452035,0,0,0,585,8120,146205
10022,1997-05-07,1998-12-31,1118755,8411.691729,10125.666281,400,91971,52006.000000,14039.885495,592461,73138,453156,4085,231246,572376,16000,293448,1600,0,944774,0,0,115170,210,4085,54516


In [10]:
trans_all = pd.concat([trans_acc,trans_2_acc],axis=0,ignore_index=False)  

trans_all = trans_all.groupby(trans_all.index).first()

trans_all = trans_all.reset_index()

trans_all['account_id'] = trans_all['account_id'].astype('int64')

trans_all.head(10)

,account_id,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
0,1,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0
1,10,1996-08-28,1998-12-31,1040615,6670.608974,7335.994095,1100,83410,47792.211538,12152.299822,549951,0,490664,5528,161759,0,544423,328905,0,0,872983,0,0,161759,345,5528,0
2,100,1993-12-28,1998-12-31,1691619,4176.837037,5074.897975,600,84377,42188.222222,12710.169747,883222,0,808397,10002,443912,0,873220,364485,0,0,1647304,0,0,33488,825,10002,0
3,1000,1993-01-25,1998-12-31,4779772,8184.541096,11288.861871,200,102396,57893.123288,18120.689825,2435539,530641,1813592,15443,502902,2413796,6300,1841331,0,0,4359262,191084,0,212993,990,15443,0
4,10001,1995-05-28,1998-12-31,1454586,5865.266129,7311.353622,600,101113,56285.205645,15714.905405,766746,0,687840,9398,15870,0,757348,671970,0,0,1428748,0,0,0,570,9398,15870
5,10005,1997-11-29,1998-12-31,969769,13284.506849,14843.203395,200,87393,46756.616438,18956.738748,511790,18130,439849,2276,111629,509314,200,346350,0,0,855744,0,0,82359,120,2276,29270
6,10018,1995-07-23,1998-12-31,1784912,5475.190184,7617.888148,-4953,68876,23585.634969,12684.444374,906592,0,878320,6760,0,0,899832,878320,0,0,1511232,0,80,266300,540,6760,0
7,10019,1995-06-15,1998-12-12,2606945,10427.780000,13185.943727,600,138173,52650.048000,27465.267872,1314297,199558,1093090,8120,146205,0,1306177,1146443,0,0,2452035,0,0,0,585,8120,146205
8,1002,1996-09-17,1998-12-31,264413,1592.849398,1841.461951,500,29756,17292.879518,5206.529453,143873,0,120540,2027,62810,0,141846,57730,0,0,223270,10340,0,28446,330,2027,0
9,10022,1997-05-07,1998-12-31,1118755,8411.691729,10125.666281,400,91971,52006.000000,14039.885495,592461,73138,453156,4085,231246,572376,16000,293448,1600,0,944774,0,0,115170,210,4085,54516


In [11]:
account_df = pd.merge(account_df, trans_all, on='account_id', how='left')

account_df.head()

#Transactions added

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0


In [12]:
dist_df = pd.read_csv("district.csv")

dist_df = dist_df.drop(columns = ['A2','A3']) 

dist_df.head()



,district_id,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,2,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674
2,3,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813
3,4,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892
4,5,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040


In [13]:
account_df = pd.merge(account_df, dist_df, on='district_id', how='left')

account_df.head()

#District Complete, now merge card and client onto disp

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0,107870,84,29,6,1,6,58.0,8754,3.8,4.31,137,3804.0,3868
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0,58796,22,16,7,1,5,51.9,9045,3.1,3.60,124,1845.0,1879


In [14]:
disp_df = pd.read_csv("disp.csv")
client_df = pd.read_csv("client.csv")
card_df = pd.read_csv("card.csv",parse_dates=[3])

card_df.head()

,card_id,disp_id,type,issued
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [15]:
disp_df = pd.merge(disp_df, client_df, on='client_id',how='left')
disp_df = pd.merge(disp_df, card_df, on='disp_id',how='left')

disp_df_all = disp_df.groupby('account_id').first()

disp_df_all = disp_df_all.drop(columns = ['disp_id','client_id','district_id','card_id']) 

disp_df_all = disp_df_all.reset_index()

disp_df_all['account_id'] = disp_df_all['account_id'].astype('int64')

disp_df_all['type_y'] = disp_df_all['type_y'].fillna('No Card')

disp_df_all.head()

,account_id,type_x,gender,birth_date,type_y,issued
0,1,OWNER,F,1970-12-13,No Card,NaT
1,2,OWNER,M,1945-02-04,No Card,NaT
2,3,OWNER,M,1956-12-01,No Card,NaT
3,4,OWNER,M,1919-09-22,No Card,NaT
4,5,OWNER,M,1929-01-25,No Card,NaT


In [16]:
account_df = pd.merge(account_df, disp_df_all, on='account_id',how='left')

account_df.head()

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,type_x,gender,birth_date,type_y,issued
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,OWNER,F,1970-12-13,No Card,NaT
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,OWNER,M,1945-02-04,No Card,NaT
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040,OWNER,M,1956-12-01,No Card,NaT
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0,107870,84,29,6,1,6,58.0,8754,3.8,4.31,137,3804.0,3868,OWNER,M,1919-09-22,No Card,NaT
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0,58796,22,16,7,1,5,51.9,9045,3.1,3.60,124,1845.0,1879,OWNER,M,1929-01-25,No Card,NaT


In [17]:
account_df.shape

(4500, 54)

In [42]:
#Merge onto our loans

loans_df = pd.read_csv("loan.csv",parse_dates=[2])

loans_base_df = pd.merge(loans_df, account_df, on='account_id',how='left')



loans_base_df.head()


,loan_id,account_id,date_x,amount,duration,payments,status,district_id,frequency,date_y,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,type_x,gender,birth_date,type_y,issued
0,4959,2,1994-01-05,80952,24,3373.0,A,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,OWNER,M,1945-02-04,No Card,NaT
1,4961,19,1996-04-29,30276,12,2523.0,B,21,POPLATEK MESICNE,1995-04-07,2523.2,0.0,0.0,0.0,0.0,2523.2,1995-04-07,1998-12-31,1575532,5199.775578,6866.583331,-10605,58158,16253.069307,12127.947343,793196,32194,750142,5589,20184,741807,45800,762152,0,0,1068101,0,58,481000,600,5589,20184,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354,OWNER,F,1939-04-23,No Card,NaT
2,4962,25,1997-12-08,30276,12,2523.0,A,68,POPLATEK MESICNE,1996-07-28,10614.2,0.0,1110.0,164.0,6817.0,2523.2,1996-07-28,1998-12-31,2958556,10797.649635,15178.207418,900,134210,56279.237226,24090.944026,1494375,85758,1378423,6203,248733,0,1488172,1215448,0,0,2733200,4428,0,184059,390,6203,30276,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887,OWNER,M,1962-02-09,No Card,NaT
3,4967,37,1998-10-14,318480,60,5308.0,D,20,POPLATEK MESICNE,1997-08-18,10305.5,0.0,2584.0,97.0,2317.0,5307.5,1997-08-18,1998-12-31,948159,7293.530769,11915.486863,-1011,104762,37547.484615,23823.350618,497029,41655,409475,2535,75280,0,494494,375850,0,0,906325,1358,0,32438,195,2535,5308,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542,OWNER,M,1952-08-26,No Card,NaT
4,4968,38,1998-04-19,110736,48,2307.0,C,19,POPLATEK TYDNE,1997-08-08,9666.8,0.0,231.0,228.0,6901.0,2306.8,1997-08-08,1998-12-31,571952,4399.630769,5450.454498,13841,55991,34462.384615,9455.331494,308266,37819,225867,2506,107007,256060,49700,156679,0,0,465082,2736,0,82812,360,2506,18456,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099,OWNER,F,1940-01-30,No Card,NaT


In [41]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_var = ['amount_order','k_symbol_LEASING',
            'k_symbol_Other', 'k_symbol_POJISTNE',
            'k_symbol_SIPO', 'k_symbol_UVER','amount_sum',
            'amount_mean', 'amount_std', 
            'balance_min', 'balance_max', 'balance_mean', 'balance_std',
            'type_PRIJEM_sum', 'type_VYBER_sum',
            'type_VYDAJ_sum', 'operation_Other_sum', 'operation_PREVOD NA UCET_sum',
            'operation_PREVOD Z UCTU_sum', 'operation_VKLAD_sum',
            'operation_VYBER_sum', 'operation_VYBER KARTOU_sum',
            'k_symbol_DUCHOD_sum', 'k_symbol_Other_sum', 'k_symbol_POJISTNE_sum',
            'k_symbol_SANKC. UROK_sum', 'k_symbol_SIPO_sum', 'k_symbol_SLUZBY_sum',
            'k_symbol_UROK_sum', 'k_symbol_UVER_sum', 'A4', 'A5', 'A6', 'A7', 'A8',
            'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']

ord_var = ['gender','frequency','type_y']
date_var = ['date_x','date_y', 'date_min', 'date_max','birth_date']
drop_var = ['amount_x','duration','payments','loan_id','account_id','district_id','type_x','issued']
#pass_col = 

loans_base_df = loans_base_df.replace([np.inf, -np.inf], np.nan)

target_raw = loans_base_df['status']
target_raw[(target_raw == 'A') | (target_raw == 'C')] = 1
target_raw[(target_raw == 'B')  | (target_raw == 'D')] = 0
#Build a column transforrner

numeric_transformer = SimpleImputer(missing_values = np.nan, strategy='median')
  
binary_transformer = OneHotEncoder(handle_unknown='ignore')

ordinal_transformer = OneHotEncoder(categories  = [ ['M','F'],
                                                     ['POPLATEK PO OBRATU','POPLATEK TYDNE','POPLATEK MESICNE'],
                                                     ['No Card','junior','classic','gold']])

preprocessor = ColumnTransformer(transformers = [ ('numerical', numeric_transformer, num_var),
                                                  ('ordinal',ordinal_transformer, ord_var)],
                                                  #('date','passthrough', date_var)],
                                                   remainder = 'drop')

X = loans_base_df.drop(columns = 'status')
y = target_raw 


#Save X, Y

loans_base_df.to_csv('loan_merged.csv')


X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state = 42)


/home/dan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/dan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
loans_base_df['status']
loans_base_df.to_csv('loan_merged.csv')

In [20]:
X_train_transformed = preprocessor.fit_transform(X_train)

X_val_transformed = preprocessor.fit_transform(X_val)

y_val = y_val.astype(bool)


In [21]:

ord_var_list = ['M','F'] +  ['POPLATEK PO OBRATU','POPLATEK TYDNE','POPLATEK MESICNE'] + ['No Card','junior','classic','gold']

transformed_columns = num_var +  ord_var_list # + date_var

print(X_train_transformed.shape)
print(len(transformed_columns))

X_train_transform_df = pd.DataFrame(X_train_transformed, columns = transformed_columns)

X_train_transform_df.head()

(545, 52)
52


,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,M,F,POPLATEK PO OBRATU,POPLATEK TYDNE,POPLATEK MESICNE,No Card,junior,classic,gold
0,6678.7,0.0,0.0,0.0,0.0,6678.7,3976149.0,14049.996466,16293.266594,500.0,127567.0,63984.127208,20995.372905,2016160.0,155325.0,1804664.0,12060.0,207049.0,2003600.0,500.0,1727340.0,25600.0,0.0,3756425.0,0.0,0.0,0.0,615.0,12060.0,207049.0,145688.0,22.0,41.0,13.0,2.0,5.0,40.7,8544.0,1.2,1.86,109.0,2719.0,2906.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,7843.0,0.0,912.0,3103.0,3195.0,633.0,1539421.0,3791.677340,5105.493284,500.0,48157.0,27463.756158,7269.835797,782159.0,0.0,757262.0,5262.0,339642.0,0.0,776897.0,417620.0,0.0,0.0,1234897.0,139635.0,0.0,143775.0,660.0,5262.0,15192.0,122603.0,25.0,21.0,6.0,2.0,8.0,80.0,8991.0,1.3,2.01,128.0,5198.0,5273.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,3160.7,0.0,0.0,0.0,0.0,3160.7,4306964.0,10353.278846,13694.289980,-670.0,138621.0,47080.875000,27197.536554,2208028.0,479514.0,1619422.0,16992.0,142245.0,0.0,2191036.0,1956691.0,0.0,0.0,4146950.0,0.0,12.0,0.0,765.0,16992.0,142245.0,323870.0,0.0,0.0,0.0,1.0,1.0,100.0,10673.0,4.7,5.44,100.0,18782.0,18347.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,16476.0,0.0,0.0,0.0,13691.0,2785.0,2804883.0,12577.950673,14677.435013,700.0,109016.0,39265.506726,17914.326139,1415124.0,104538.0,1285221.0,4735.0,480121.0,1402089.0,8300.0,909638.0,0.0,0.0,2317027.0,0.0,0.0,424421.0,3000.0,4735.0,55700.0,58796.0,22.0,16.0,7.0,1.0,5.0,51.9,9045.0,3.1,3.60,124.0,1845.0,1879.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,7485.0,0.0,4235.0,0.0,1978.0,1272.0,2173132.0,4734.492375,6214.145313,900.0,57224.0,33080.690632,9437.582439,1102588.0,0.0,1070544.0,8596.0,408984.0,0.0,1093992.0,661560.0,0.0,0.0,1991872.0,0.0,0.0,110768.0,840.0,8596.0,61056.0,1204953.0,0.0,0.0,0.0,1.0,1.0,100.0,12541.0,0.2,0.43,167.0,85677.0,99107.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [22]:
X_train_transform_df.isna().sum()

amount_order                    0
k_symbol_LEASING                0
k_symbol_Other                  0
k_symbol_POJISTNE               0
k_symbol_SIPO                   0
k_symbol_UVER                   0
amount_sum                      0
amount_mean                     0
amount_std                      0
balance_min                     0
balance_max                     0
balance_mean                    0
balance_std                     0
type_PRIJEM_sum                 0
type_VYBER_sum                  0
type_VYDAJ_sum                  0
operation_Other_sum             0
operation_PREVOD NA UCET_sum    0
operation_PREVOD Z UCTU_sum     0
operation_VKLAD_sum             0
operation_VYBER_sum             0
operation_VYBER KARTOU_sum      0
k_symbol_DUCHOD_sum             0
k_symbol_Other_sum              0
k_symbol_POJISTNE_sum           0
k_symbol_SANKC. UROK_sum        0
k_symbol_SIPO_sum               0
k_symbol_SLUZBY_sum             0
k_symbol_UROK_sum               0
k_symbol_UVER_

In [23]:
from sklearn.ensemble import RandomForestClassifier
import eli5


rf_model = RandomForestClassifier(class_weight = 'balanced',oob_score= True, random_state = 42)

rf_model.fit(X_train_transform_df, y_train.astype(bool))

eli5.show_weights(rf_model, feature_names = transformed_columns)

/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.3589 ± 0.7151,balance_min
0.2614 ± 0.5973,k_symbol_SANKC. UROK_sum
0.0470 ± 0.1597,balance_mean
0.0457 ± 0.1655,k_symbol_SIPO
0.0278 ± 0.0986,balance_std
0.0166 ± 0.0757,operation_PREVOD NA UCET_sum
0.0157 ± 0.0704,amount_mean
0.0141 ± 0.0607,balance_max
0.0136 ± 0.0719,k_symbol_UVER
0.0125 ± 0.0665,amount_std


In [24]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
     'ccp_alpha': [ 0.0, 0.1 ],
     'max_depth': [5, 10, None],
     'max_features': ['auto','log2'],
     'max_leaf_nodes': [5,10,None],
     'min_impurity_decrease': [ 0.0, 0.1 ],
     'min_samples_leaf': [ 1, 2, 4],
     'min_samples_split': [ 2, 3, 4],
     'min_weight_fraction_leaf': [0.0, 0.1, 0.2 ],
     'n_estimators': [50 , 80, 110, 140, 200]
}

grid_search = RandomizedSearchCV(rf_model, param_grid, cv=5, verbose=1,n_iter=100)
grid_search.fit(X_train_transform_df, y_train.astype(bool))

print(("best RF from grid search: %.3f"
       % grid_search.score(X_val_transformed, y_val)))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


best RF from grid search: 0.993


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.2min finished


In [25]:
results  = pd.DataFrame(grid_search.cv_results_)

In [26]:
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_weight_fraction_leaf,param_min_samples_split,param_min_samples_leaf,param_min_impurity_decrease,param_max_leaf_nodes,param_max_features,param_max_depth,param_ccp_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
45,0.147428,0.001702,0.007503,0.000126,110,0,2,2,0,5,log2,None,0.1,"{'n_estimators': 110, 'min_weight_fraction_lea...",1.000000,0.990826,1.000000,1.000000,1.0,0.998165,0.003670,1
51,0.120644,0.001995,0.005876,0.000220,80,0,2,4,0,None,auto,None,0.1,"{'n_estimators': 80, 'min_weight_fraction_leaf...",1.000000,0.990826,1.000000,1.000000,1.0,0.998165,0.003670,1
68,0.159434,0.013617,0.007557,0.000165,110,0,2,1,0,10,log2,10,0.1,"{'n_estimators': 110, 'min_weight_fraction_lea...",1.000000,0.990826,1.000000,1.000000,1.0,0.998165,0.003670,1
60,0.116919,0.002972,0.005928,0.000203,80,0,2,4,0,None,auto,10,0.1,"{'n_estimators': 80, 'min_weight_fraction_leaf...",1.000000,0.990826,1.000000,1.000000,1.0,0.998165,0.003670,1
54,0.107398,0.001591,0.005685,0.000116,80,0,3,4,0,5,log2,None,0.1,"{'n_estimators': 80, 'min_weight_fraction_leaf...",1.000000,0.990826,0.990826,1.000000,1.0,0.996330,0.004494,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,0.288897,0.019733,0.013963,0.001196,200,0.2,2,4,0,10,log2,5,0.1,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.981651,0.990826,0.954128,0.990826,1.0,0.983486,0.015784,92
62,0.258850,0.003482,0.013200,0.000528,200,0,3,4,0.1,10,auto,None,0.1,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.981651,0.990826,0.954128,0.990826,1.0,0.983486,0.015784,92
13,0.168482,0.006411,0.009556,0.000496,140,0.2,4,2,0.1,5,log2,5,0,"{'n_estimators': 140, 'min_weight_fraction_lea...",0.981651,0.981651,0.954128,0.990826,1.0,0.981651,0.015352,98
86,0.250641,0.005926,0.013292,0.000949,200,0,3,4,0.1,5,log2,10,0.1,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.981651,0.981651,0.954128,0.990826,1.0,0.981651,0.015352,98


In [27]:
y_pred_val = grid_search.predict(X_val_transformed)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred_val)

array([[ 15,   1],
       [  0, 121]])

In [28]:
eli5.show_weights(grid_search.best_estimator_, feature_names = transformed_columns)

Weight,Feature
0.4930 ± 0.7966,balance_min
0.3252 ± 0.7082,k_symbol_SANKC. UROK_sum
0.0628 ± 0.3331,k_symbol_SIPO
0.0340 ± 0.2145,balance_mean
0.0112 ± 0.0564,operation_PREVOD Z UCTU_sum
0.0106 ± 0.0666,balance_std
0.0085 ± 0.0650,operation_VKLAD_sum
0.0074 ± 0.0528,amount_std
0.0064 ± 0.0525,amount_mean
0.0057 ± 0.0450,No Card


In [29]:
#Apply Model to other accounts, then see how they cluster...

grid_search.best_params_

{'n_estimators': 110,
 'min_weight_fraction_leaf': 0.0,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'min_impurity_decrease': 0.0,
 'max_leaf_nodes': 5,
 'max_features': 'log2',
 'max_depth': None,
 'ccp_alpha': 0.1}

In [30]:
account_transformed = preprocessor.fit_transform(account_df)

account_transform_df = pd.DataFrame(account_transformed, columns = transformed_columns)

In [31]:
account_transform_df['account_id'] = account_df['account_id']

In [32]:
account_transform_df.shape

(4500, 53)

In [33]:
loans_base_df.shape

(682, 60)

In [34]:
(~account_transform_df['account_id'].isin(loans_base_df['account_id'])).sum()

3818

In [35]:
account_no_loan_df = account_transform_df[(~account_transform_df['account_id'].isin(loans_base_df['account_id']))]

account_no_loan_df = account_no_loan_df.drop(columns='account_id')

account_no_loan_df.head()

,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,M,F,POPLATEK PO OBRATU,POPLATEK TYDNE,POPLATEK MESICNE,No Card,junior,classic,gold
0,2452.0,0.0,0.0,0.0,2452.0,0.0,375192.0,1569.841004,1732.115246,1000.0,30415.0,15994.079498,3656.858500,194322.0,0.0,180870.0,3067.0,98080.0,165555.0,25700.0,82790.0,0.0,0.0,273445.0,0.0,0.0,98080.0,600.0,3067.0,0.0,70699.0,60.0,13.0,2.0,1.0,4.0,65.3,8968.0,2.8,3.35,131.0,1740.0,1910.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,5001.0,0.0,327.0,3539.0,1135.0,0.0,295030.0,2521.623932,2951.300633,1000.0,53447.0,26973.529915,10508.728242,173062.0,0.0,121968.0,2073.0,65013.0,0.0,170989.0,56955.0,0.0,0.0,232000.0,46007.0,0.0,14755.0,195.0,2073.0,0.0,95616.0,65.0,30.0,4.0,1.0,6.0,51.4,9307.0,3.8,4.43,118.0,2616.0,3040.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3363.0,0.0,0.0,0.0,3363.0,0.0,350986.0,1887.021505,1956.829637,800.0,34870.0,21753.526882,5009.976472,192349.0,0.0,158637.0,2747.0,98812.0,188802.0,800.0,59825.0,0.0,188802.0,98740.0,0.0,0.0,60262.0,435.0,2747.0,0.0,107870.0,84.0,29.0,6.0,1.0,6.0,58.0,8754.0,3.8,4.31,137.0,3804.0,3868.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,2668.0,0.0,0.0,0.0,2668.0,0.0,166891.0,1986.797619,1981.162764,600.0,32036.0,25090.023810,4752.452095,97489.0,0.0,69402.0,1566.0,37352.0,95323.0,600.0,32050.0,0.0,95323.0,32440.0,0.0,0.0,37352.0,210.0,1566.0,0.0,58796.0,22.0,16.0,7.0,1.0,5.0,51.9,9045.0,3.1,3.60,124.0,1845.0,1879.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5,3954.0,0.0,0.0,0.0,3954.0,0.0,647584.0,2632.455285,2655.001376,900.0,51879.0,34547.353659,7331.711741,347616.0,0.0,299968.0,6597.0,185838.0,340119.0,900.0,114130.0,0.0,340119.0,114340.0,0.0,0.0,185838.0,690.0,6597.0,0.0,121947.0,37.0,28.0,7.0,3.0,11.0,70.5,8541.0,2.5,2.97,131.0,3496.0,3839.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [36]:
y_acc_pred_val = grid_search.best_estimator_.predict(account_no_loan_df)


In [37]:
no_loans_df = account_no_loan_df[y_acc_pred_val==0]

preapp_loans_df = account_no_loan_df[y_acc_pred_val==1]

print(no_loans_df.shape)
print(preapp_loans_df.shape)


(193, 52)
(3625, 52)


In [38]:
X_zero = no_loans_df
X_one = preapp_loans_df

A = (X_one.describe().T - X_zero.describe().T)

print(A.sort_values(by ='mean', ascending=False))

                               count          mean            std  \
operation_PREVOD NA UCET_sum  3432.0  1.139044e+05   34391.584401   
k_symbol_SIPO_sum             3432.0  4.837881e+04  -21274.135621   
k_symbol_DUCHOD_sum           3432.0  4.215426e+04   71876.399715   
balance_min                   3432.0  5.610280e+03   -5528.008814   
operation_VYBER KARTOU_sum    3432.0  2.658854e+03    8140.028187   
k_symbol_SIPO                 3432.0  1.164777e+03     826.282228   
k_symbol_Other                3432.0  2.726925e+02     172.211524   
A5                            3432.0  1.710188e+00      -1.114954   
A6                            3432.0  4.254922e-01      -1.087228   
A9                            3432.0  1.930120e-01      -0.117897   
POPLATEK MESICNE              3432.0  1.658145e-01      -0.199241   
A13                           3432.0  1.160951e-01      -0.093424   
A12                           3432.0  9.052035e-02      -0.093909   
F                             3432

In [39]:
# Do Some SHAP value!?!?!?!